## Примеры работы с pylatex


In [ ]:
%pip install pylatex

In [ ]:
from pylatex import Document, Section, Subsection, Command
from pylatex.utils import italic, NoEscape


def fill_document(doc):
    """Add a section, a subsection and some text to the document.

    :param doc: the document
    :type doc: :class:`pylatex.document.Document` instance
    """
    with doc.create(Section("A section")):
        doc.append("Some regular text and some ")
        doc.append(italic("italic text. "))

        with doc.create(Subsection("A subsection")):
            doc.append("Also some crazy characters: $&#{}")


### Basic document


In [ ]:
# Basic document
doc = Document("basic")
fill_document(doc)

doc.generate_pdf(clean_tex=False)
doc.generate_tex()

# Document with `\maketitle` command activated
doc = Document()

doc.preamble.append(Command("title", "Awesome Title"))
doc.preamble.append(Command("author", "Anonymous author"))
doc.preamble.append(Command("date", NoEscape(r"\today")))
doc.append(NoEscape(r"\maketitle"))

fill_document(doc)

doc.generate_pdf("basic_maketitle", clean_tex=False)

# Add stuff to the document
with doc.create(Section("A second section")):
    doc.append("Some text.")

doc.generate_pdf("basic_maketitle2", clean_tex=False)
tex = doc.dumps()  # The document as string in LaTeX syntax


### Multirow example


In [ ]:
from pylatex import Document, Section, Subsection, Tabular, MultiColumn, MultiRow

doc = Document("multirow")
section = Section("Multirow Test")

test1 = Subsection("MultiColumn")
test2 = Subsection("MultiRow")
test3 = Subsection("MultiColumn and MultiRow")
test4 = Subsection("Vext01")

table1 = Tabular("|c|c|c|c|")
table1.add_hline()
table1.add_row((MultiColumn(4, align="|c|", data="Multicolumn"),))
table1.add_hline()
table1.add_row((1, 2, 3, 4))
table1.add_hline()
table1.add_row((5, 6, 7, 8))
table1.add_hline()
row_cells = ("9", MultiColumn(3, align="|c|", data="Multicolumn not on left"))
table1.add_row(row_cells)
table1.add_hline()

table2 = Tabular("|c|c|c|")
table2.add_hline()
table2.add_row((MultiRow(3, data="Multirow"), 1, 2))
table2.add_hline(2, 3)
table2.add_row(("", 3, 4))
table2.add_hline(2, 3)
table2.add_row(("", 5, 6))
table2.add_hline()
table2.add_row((MultiRow(3, data="Multirow2"), "", ""))
table2.add_empty_row()
table2.add_empty_row()
table2.add_hline()

table3 = Tabular("|c|c|c|")
table3.add_hline()
table3.add_row(
    (MultiColumn(2, align="|c|", data=MultiRow(2, data="multi-col-row")), "X")
)
table3.add_row((MultiColumn(2, align="|c|", data=""), "X"))
table3.add_hline()
table3.add_row(("X", "X", "X"))
table3.add_hline()

table4 = Tabular("|c|c|c|")
table4.add_hline()
col1_cell = MultiRow(4, data="span-4")
col2_cell = MultiRow(2, data="span-2")
table4.add_row((col1_cell, col2_cell, "3a"))
table4.add_hline(start=3)
table4.add_row(("", "", "3b"))
table4.add_hline(start=2)
table4.add_row(("", col2_cell, "3c"))
table4.add_hline(start=3)
table4.add_row(("", "", "3d"))
table4.add_hline()

test1.append(table1)
test2.append(table2)
test3.append(table3)
test4.append(table4)

section.append(test1)
section.append(test2)
section.append(test3)
section.append(test4)

doc.append(section)
doc.generate_pdf(clean_tex=False)


## Генерация таблицы по актам


In [ ]:
%pip install jira
%pip install dataclasses-json

In [ ]:
from datetime import datetime
from jira_dtos import UserWorklogItemDto


t = UserWorklogItemDto(
    task_code="2,",
    task_name="kekw",
    start_period_date=datetime.now(),
    end_period_date=datetime.now(),
    time_spent_seconds=234234,
)

print(t)


In [ ]:
from jira import JIRA
import json

from jira_dtos import JiraInfo, DateRange
from jira_helpers import map_user_worklog, map_issues


In [ ]:
# NOTE: a sensitive data
user_login = "JIRA LOGIN"
user_jira_password = "PASSWORD"
jira_info = JiraInfo(
    jira_server_address="https://jira.com/",
    basic_auth=(user_login, user_jira_password),
    user_login=user_login,
)


In [ ]:
auth_jira = JIRA(
    basic_auth=jira_info.basic_auth, options={"server": jira_info.jira_server_address}
)


In [ ]:
startWorklogDate = "2022-08-14"
endWorklogDate = "2022-09-14"
jql = f'worklogAuthor in ("{jira_info.user_login}") and worklogDate >= {startWorklogDate} and worklogDate < {endWorklogDate}'
found_issues = auth_jira.search_issues(jql)
print(found_issues)


In [ ]:
from time import strptime
import dateutil.parser

start_date = dateutil.parser.isoparse(startWorklogDate)
end_date = dateutil.parser.isoparse(endWorklogDate)
worklogs_data_range = DateRange(start_date=start_date, end_date=end_date)
map_issue = lambda issue: map_issues(issue, jira_info.user_login, worklogs_data_range)
user_worklogs = map(map_issue, found_issues)
user_worklog_items = list(map(map_user_worklog, user_worklogs))


In [ ]:
t1 = UserWorklogItemDto.schema().dumps(user_worklog_items, many=True)
print(t1)
t2 = UserWorklogItemDto.schema().loads(t1, many=True)
t3 = list(filter(None, t2))
print()
print(t3[0].task_code)


In [ ]:
in_hours = filter(lambda x: x != None, user_worklog_items)
in_hours = map(lambda x: x.time_spent_seconds, in_hours)  # type: ignore
s = sum(in_hours)
print(s / 3600)


In [ ]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json


@dataclass_json
@dataclass
class Person:
    name: str


people_json = [Person("lidatong")]
t = Person.schema().dumps(people_json, many=True)  # '[{"name": "lidatong"}]'
t2 = Person.schema().loads('[{"name2": "lidatong"}]', many=True)

print(type(t2[0]))


In [ ]:
# generate_act_doc(user_worklog_items)


## Пример работы с питоновскими датами -> str и обратно


In [ ]:
%pip install python-dateutil

In [ ]:
import dateutil.parser

raw_date = "2022-08-14T15:59:00.000+0300"
date = dateutil.parser.isoparse(raw_date)
print(date)
print(type(date))

str_date = date.isoformat()
print(str_date)

str2_date = "2022-08-14"
date2 = dateutil.parser.isoparse(str2_date)
print(date2)

is_intersected_yep = date2.timestamp() <= date.timestamp()
print(is_intersected_yep)


## Пример генерации PDF актов через pylatex


In [ ]:
%pip install pylatex

In [ ]:
from dataclasses import dataclass
from typing import final
from pylatex import (
    Document,
    Section,
    Subsection,
    Tabular,
    MultiColumn,
    MultiRow,
    Command,
)
from jira_helpers import MappedUserWorklog
from jira_dtos import UserWorklogItemDto
from pylatex.utils import bold


@final
@dataclass
class ActRecord:
    column_number: int
    service_name: str
    time_spent_in_hours: float


MappedUserWorklogs = list[MappedUserWorklog]
ActRecords = list[ActRecord]


def map_act_record(worklog: UserWorklogItemDto, index: int) -> ActRecord:
    service_name = f"{worklog.task_code}. {worklog.task_name}"
    in_hours = worklog.time_spent_seconds / 3600
    record = ActRecord(
        column_number=index, service_name=service_name, time_spent_in_hours=in_hours
    )

    return record


def format_act_records(worklogs: MappedUserWorklogs) -> ActRecords:
    filtered = filter(None, worklogs)
    enumerated = enumerate(filtered)
    act_records = map(lambda x: map_act_record(x[1], x[0]), enumerated)
    act_records = list(act_records)

    return act_records


In [ ]:
from dataclasses import dataclass
from typing import final
from pylatex import (
    Document,
    Section,
    Subsection,
    Tabular,
    MultiColumn,
    MultiRow,
    Command,
)
from jira_helpers import MappedUserWorklog
from jira_dtos import UserWorklogItemDto
from pylatex.utils import bold


@final
@dataclass
class TaskRecord:
    column_number: int
    service_name: str
    validity: str


TaskRecords = list[TaskRecord]


def map_task_record(worklog: UserWorklogItemDto, index: int) -> TaskRecord:
    service_name = f"{worklog.task_code}. {worklog.task_name}"
    start_period = worklog.start_period_date.strftime("%d.%m.%Y")
    end_period = worklog.end_period_date.strftime("%d.%m.%Y")
    validity = f"{start_period} - {end_period}"
    record = TaskRecord(
        column_number=index, service_name=service_name, validity=validity
    )

    return record


def format_task_records(worklogs: MappedUserWorklogs) -> TaskRecords:
    filtered = filter(None, worklogs)
    enumerated = enumerate(filtered)
    task_records = map(lambda x: map_task_record(x[1], x[0]), enumerated)
    task_records = list(task_records)

    return task_records


In [ ]:
from datetime import datetime

time = datetime.now().strftime("%d.%m.%Y")
print("time:", time)


In [ ]:
def generate_act_table(worklogs: MappedUserWorklogs) -> Tabular:
    table = Tabular("| p | p | p |", row_height=1.5)
    table.add_hline()
    header = [
        "№",
        MultiColumn(size=1, align="|r|", data="Наименование услуг (объем, перечень)"),
        "Затраченное время (часы)",
    ]
    table.add_row(header, mapper=[bold])
    table.add_hline()

    formatted_records = format_act_records(worklogs)
    for record in formatted_records:
        cells = [record.column_number, record.service_name, record.time_spent_in_hours]
        table.add_row(cells)
        table.add_hline()

    row_cells = [
        MultiColumn(size=2, align="|r|", data="Стоимость, руб.: 331 703, 40 руб. "),
        173.7,
    ]
    table.add_row(row_cells, mapper=[bold])
    table.add_hline()

    return table


def generate_act_doc(worklogs) -> None:
    doc = Document("акты", page_numbers=True, font_size="13pt")
    doc.preamble.append(Command("usepackage", "babel", "russian"))
    doc.preamble.append(Command("usepackage", "graphicx"))
    doc.preamble.append(Command("usepackage", "array"))
    doc.preamble.append(Command("usepackage", "lipsum"))
    doc.preamble.append(Command("usepackage", "geometry"))
    doc.preamble.append(
        Command("geometry", "{a4paper, left=10mm, right=10mm, top=20mm")
    )

    section = Section("АКТ СДАЧИ ПРИЕМКИ")

    test1 = Subsection("Пример таблицы")
    table1 = generate_act_table(worklogs)
    test1.append(table1)
    section.append(test1)

    doc.append(section)
    doc.generate_pdf(clean_tex=False, compiler="lualatex")


In [ ]:
def generate_task_table(worklogs: MappedUserWorklogs) -> Tabular:
    table = Tabular("| p | p | p |", row_height=1.5)
    table.add_hline()
    header = [
        "№",
        MultiColumn(size=1, align="|с|", data="Наименование услуг"),
        "Период выполнения работ",
    ]
    table.add_hline()

    formatted_records = format_task_records(worklogs)
    for record in formatted_records:
        cells = [record.column_number, record.service_name, record.validity]
        table.add_row(cells)
        table.add_hline()

    row_cells = [
        MultiColumn(size=2, align="|r|", data="Стоимость, руб.: 331 703, 40 руб. "),
        173.7,
    ]
    table.add_row(row_cells, mapper=[bold])
    table.add_hline()

    return table


def generate_task_doc(worklogs) -> None:
    doc = Document("задание", page_numbers=True, font_size="13pt")
    doc.preamble.append(Command("usepackage", "babel", "russian"))
    doc.preamble.append(Command("usepackage", "graphicx"))
    doc.preamble.append(Command("usepackage", "array"))
    doc.preamble.append(Command("usepackage", "lipsum"))
    doc.preamble.append(Command("usepackage", "geometry"))
    doc.preamble.append(
        Command("geometry", "{a4paper, left=10mm, right=10mm, top=20mm")
    )

    section = Section("Задание на выполенине работ")

    test1 = Subsection("Пример таблицы")
    table1 = generate_task_table(worklogs)
    test1.append(table1)
    section.append(test1)

    doc.append(section)
    doc.generate_pdf(clean_tex=False, compiler="lualatex")


In [ ]:
worklogs_str = '[{"task_name":"Что есть зло? Чтобы это не было, оно возникает из слабости","task_code":"D2C-1","time_spent_seconds":52200,"start_period_date":1662987567.499,"end_period_date":1663078476.16},{"task_name":"Требуется много таланта и умения, чтобы скрыть свой талант и умение","task_code":"D2C-2","time_spent_seconds":126000,"start_period_date":1662473799.184,"end_period_date":1663047038.527},{"task_name":"Человек - это животное, которое умеет совершать сделки. Ни одна собака не будет меняться костью с другой собакой","task_code":"D2C-3","time_spent_seconds":225000,"start_period_date":1661522651.363,"end_period_date":1662722506.429},{"task_name":"Мы не должны расстраиваться, когда другие скрывают правду от нас, так как мы скрываем правду и от себя тоже","task_code":"D2C-4","time_spent_seconds":115200,"start_period_date":1660831746.787,"end_period_date":1661497110.483},{"task_name":"У каждого человека внутри сидит предатель","task_code":"D2C-10","time_spent_seconds":99000,"start_period_date":1660573388.942,"end_period_date":1660813309.804}]'

In [ ]:
from jira_dtos import UserWorklogItemDto


worklogs = UserWorklogItemDto.schema().loads(worklogs_str, many=True)

generate_act_doc(worklogs)


In [ ]:
from jira_dtos import UserWorklogItemDto


worklogs = UserWorklogItemDto.schema().loads(worklogs_str, many=True)

generate_task_doc(worklogs)


## Пример генерации pdf Акта через Lualatex


In [ ]:
import subprocess

file_path = "./test_acts.tex"
job_name = "--job-name=kekw"
output_directory = "--output-directory=generated_pdfs"
subprocess.run(["lualatex", job_name, output_directory, file_path])


## Формирование отчета на основе жсон данных и шаблона в Jinja


### Установка зависимостей


In [ ]:
%pip install Jinja2

In [ ]:
%pip install requests

### Работа с TEX файлами


In [ ]:
from typing import final
from dataclasses import dataclass
import jinja2
from pathlib import Path
import os


@dataclass
class JinjaTemplate:
    template: jinja2.Template
    template_name: str = ""


@final
@dataclass
class ActJinjaTemplate(JinjaTemplate):
    pass


@final
@dataclass
class TaskJinjaTemplate(JinjaTemplate):
    pass


def get_doc_template(filename: str) -> JinjaTemplate:
    templateLoader = jinja2.FileSystemLoader(searchpath="./templates/")
    templateEnv = jinja2.Environment(loader=templateLoader)
    template = templateEnv.get_template(filename)
    template_name = Path(filename).stem
    jinja_template = JinjaTemplate(template=template, template_name=template_name)

    return jinja_template


def get_file_path_in_cur_dir(file_name_path: Path) -> Path:
    current_dir = os.path.abspath("")
    path = Path(current_dir, file_name_path)

    return path


RenderedTexData = str


def get_tex_path(template_name: str) -> Path:
    file_name = f"{template_name}.tex"
    file_path = Path("generated_tex", file_name)

    return file_path


def write_tex_file(template_name: str, data: RenderedTexData) -> Path:
    file_name_path = get_tex_path(template_name)
    path = get_file_path_in_cur_dir(file_name_path)
    with open(path, "w", encoding="utf-8") as fp:
        fp.write(data)

    return path


def get_act_doc_template() -> JinjaTemplate:
    template = get_doc_template("act.jinja")
    act_template = ActJinjaTemplate(
        template=template.template, template_name=template.template_name
    )

    return act_template


def get_task_doc_template() -> JinjaTemplate:
    template = get_doc_template("task.jinja")
    act_template = TaskJinjaTemplate(
        template=template.template, template_name=template.template_name
    )

    return act_template


JinjaTemplates = list[JinjaTemplate]


In [ ]:
from abc import ABCMeta, abstractmethod
import jinja2
from jira_helpers import MappedUserWorklog
from typing import final


MappedUserWorklogs = list[MappedUserWorklog]


@final
@dataclass
class ContractInfo:
    approval_date: str = ""
    """Approval date, for example (родительный падеж):
        <<15>> августа 2022 г."""
    number: str = ""
    """Document number (day-month-year), for example: 15-08-22"""


@final
@dataclass
class WorkInfo:
    exact_work_cost: str = ""
    """Exact work cost, for example: 322 228.666"""
    exact_comma_work_cost: str = ""
    """Exact work cost, for example: 322 228, 666"""
    work_cost: str = ""
    """Work cost, for example: 322 228"""
    work_cost_in_words: str = ""
    """Work cost in words (винительный падеж), for example:
        322 228 (триста двадцать две тысячи двести двадцать восемь) рублей 00 коп."""
    total_spent_hours: float = 0.0
    """Total spent hours, for example: 176.5"""


@final
@dataclass
class ContractsInfo:
    act: ContractInfo
    contract: ContractInfo
    supplementary_agreement: ContractInfo
    work_info: WorkInfo


@final
@dataclass
class InclinedInfo:
    nominative: str = ""
    genitive: str = ""
    dative: str = ""
    accusative: str = ""
    instrumental: str = ""
    prepositional: str = ""


@final
@dataclass
class CompanyInfo:
    header_fullname: InclinedInfo
    header_fullname_initials_in_end: InclinedInfo
    header_fullname_initials_in_front: InclinedInfo
    header_position: InclinedInfo
    company_name: str
    initial_company_name: str


@final
@dataclass
class ContractPartiesInfo:
    customer: CompanyInfo
    executor: CompanyInfo


@dataclass
class ITemplateInfo(metaclass=ABCMeta):
    contract_parties_info: ContractPartiesInfo
    doc_template: JinjaTemplate
    contracts_info: ContractsInfo
    worklogs: MappedUserWorklogs

    @abstractmethod
    def get_template_data(self) -> object:
        raise NotImplementedError

    @final
    def render_template(self) -> RenderedTexData:
        template_data = self.get_template_data()
        rendered_tex_data = self.doc_template.template.render(data=template_data)

        return rendered_tex_data


def format_tex_file_by_doc_template(info: ITemplateInfo) -> Path:
    rendered_tex_data = info.render_template()
    wrotten_tex_file_path = write_tex_file(
        info.doc_template.template_name, rendered_tex_data
    )

    return wrotten_tex_file_path


### Формирование данных по Акту


In [ ]:
from dataclasses import dataclass
from typing import final
from jira_dtos import UserWorklogItemDto


@final
@dataclass
class ActRecord:
    column_number: int
    service_name: str
    time_spent_in_hours: float


ActRecords = list[ActRecord]


def map_act_record(worklog: UserWorklogItemDto, index: int) -> ActRecord:
    service_name = f"{worklog.task_code}. {worklog.task_name}"
    in_hours = worklog.time_spent_seconds / 3600
    column_number = index + 1
    record = ActRecord(
        column_number=column_number,
        service_name=service_name,
        time_spent_in_hours=in_hours,
    )

    return record


def format_act_records(worklogs: MappedUserWorklogs) -> ActRecords:
    filtered = filter(None, worklogs)
    enumerated = enumerate(filtered)
    act_records = map(lambda x: map_act_record(x[1], x[0]), enumerated)
    act_records = list(act_records)

    return act_records


@final
@dataclass
class ActReport:
    act: ContractInfo
    contract: ContractInfo
    supplementary_agreement: ContractInfo
    work_info: WorkInfo
    act_records: ActRecords
    contract_parties_info: ContractPartiesInfo


def format_act_report(data: ITemplateInfo) -> ActReport:
    act_records = format_act_records(data.worklogs)
    report = ActReport(
        act=data.contracts_info.act,
        contract=data.contracts_info.contract,
        supplementary_agreement=data.contracts_info.supplementary_agreement,
        work_info=data.contracts_info.work_info,
        act_records=act_records,
        contract_parties_info=data.contract_parties_info,
    )

    return report


@final
@dataclass
class ActData(ITemplateInfo):
    def get_template_data(self) -> object:
        return format_act_report(self)


### Формирование данных по Заданию


In [ ]:
from dataclasses import dataclass
from typing import Any, final
from pylatex import (
    Document,
    Section,
    Subsection,
    Tabular,
    MultiColumn,
    MultiRow,
    Command,
)
from jira_helpers import MappedUserWorklog
from jira_dtos import UserWorklogItemDto
from pylatex.utils import bold


@final
@dataclass
class TaskRecord:
    column_number: int
    service_name: str
    validity: str


TaskRecords = list[TaskRecord]


def map_task_record(worklog: UserWorklogItemDto, index: int) -> TaskRecord:
    service_name = f"{worklog.task_code}. {worklog.task_name}"
    start_period = worklog.start_period_date.strftime("%d.%m.%y")
    end_period = worklog.end_period_date.strftime("%d.%m.%y")
    validity = f"{start_period} – {end_period}"
    index = index + 1
    record = TaskRecord(
        column_number=index, service_name=service_name, validity=validity
    )

    return record


def format_task_records(worklogs: MappedUserWorklogs) -> TaskRecords:
    filtered = filter(None, worklogs)
    enumerated = enumerate(filtered)
    task_records = map(lambda x: map_task_record(x[1], x[0]), enumerated)
    task_records = list(task_records)

    return task_records


@final
@dataclass
class TaskReport:
    task: ContractInfo
    contract: ContractInfo
    supplementary_agreement: ContractInfo
    work_info: WorkInfo
    task_records: TaskRecords
    contract_parties_info: ContractPartiesInfo


def format_task_report(data: ITemplateInfo) -> TaskReport:
    task_records = format_task_records(data.worklogs)
    report = TaskReport(
        task=data.contracts_info.supplementary_agreement,
        contract=data.contracts_info.contract,
        supplementary_agreement=data.contracts_info.supplementary_agreement,
        work_info=data.contracts_info.work_info,
        task_records=task_records,
        contract_parties_info=data.contract_parties_info,
    )

    return report


@final
@dataclass
class TaskData(ITemplateInfo):
    def get_template_data(self) -> object:
        return format_task_report(self)


In [ ]:
import subprocess
from pathlib import Path

FilePath = str


def generate_pdf_acts(tex_file_path: Path) -> Path:
    tex_file_name = tex_file_path.stem
    job_name = f"--job-name={tex_file_name}"
    generated_pdf_folder = "generated_pdfs"
    output_directory = f"--output-directory={generated_pdf_folder}"
    tex_file_pdf_folder = Path(generated_pdf_folder, tex_file_name)
    generated_pdf_file_path = get_file_path_in_cur_dir(tex_file_pdf_folder)
    text_file_filesystem_path = str(tex_file_path)
    t = [job_name, output_directory, text_file_filesystem_path]
    print(t)
    subprocess.run(["lualatex", job_name, output_directory, text_file_filesystem_path])

    return generated_pdf_file_path


### Склонение слов


#### Склонение стоимости


In [ ]:
import requests
import json
import sys
from dataclasses import dataclass
from typing import final


@final
@dataclass
class CostInfo:
    cost: float = 0.0
    unit: str = "рубль"


@final
@dataclass
class InclinedCost:
    cost: str = ""
    unit: str = ""


InclinedCostResult = InclinedCost | None


def incline_cost(cost_info: CostInfo) -> InclinedCostResult:
    try:
        url = "https://ws3.morpher.ru/russian/spell"
        headers = {"User-Agent": "My Python script"}
        number_str = str(cost_info.cost)
        params = dict(
            n=number_str,
            unit=cost_info.unit,
            format="json",
            # token= "AUTH TOKEN HERE"
        )

        response = requests.get(url=url, params=params, headers=headers)
        is_failed_response = response.status_code != 200
        if is_failed_response:
            print(
                f"Failed response({response.status_code}) to morpher on do incline cost",
                file=sys.stderr,
            )

            return None

        data = json.loads(response.text)
        cost = data.get("n").get("В")
        unit = data.get("unit").get("В")
        inclined_cost = InclinedCost(cost=cost, unit=unit)

        return inclined_cost
    except Exception:
        print("Unknown error on do incline cost", file=sys.stderr)

        return None


In [ ]:
cost_info = CostInfo(cost=322228, unit="рубль")
test_inclined_cost = incline_cost(cost_info)
if test_inclined_cost is None:
    print("Failed get inclined cost")
else:
    print(
        f"Inclined cost (винительный): {test_inclined_cost.cost} {test_inclined_cost.unit}"
    )


#### Склонение месяца


In [ ]:
import requests

MonthName = str

InclinedMonthResult = MonthName | None


def incline_month(month_name: MonthName) -> InclinedMonthResult:
    try:
        url = "https://ws3.morpher.ru/russian/declension"
        headers = {"User-Agent": "My Python script"}
        params = dict(
            s=month_name,
            format="json",
            # token= "AUTH TOKEN HERE"
        )

        response = requests.get(url=url, params=params, headers=headers)
        is_failed_response = response.status_code != 200
        if is_failed_response:
            print(
                f"Failed response({response.status_code}) to morpher on do incline month",
                file=sys.stderr,
            )

            return None

        data = json.loads(response.text)
        inclined_month = data.get("Р")

        return inclined_month
    except Exception:
        print("Unknown error on do incline month", file=sys.stderr)

        return None


In [ ]:
test_inclined_month = incline_month("август")
if test_inclined_month is None:
    print("Failed get inclined month")
else:
    print(f"Inclined month (родительный): {test_inclined_month}")


#### Склонение ФИО


In [ ]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json, Undefined
from typing import Optional, final


@dataclass_json
@dataclass
class FullnameInfo:
    Ф: str = ""
    И: str = ""
    О: str = ""


@dataclass_json
@dataclass
class InclinedMorpherDeclensionMany:
    И: str = ""
    Р: str = ""
    Д: str = ""
    В: str = ""
    Т: str = ""
    П: str = ""


@dataclass_json
@dataclass
class InclinedMorpherInfo:
    Р: str = ""
    Д: str = ""
    В: str = ""
    Т: str = ""
    П: str = ""
    ФИО: Optional[FullnameInfo] = None
    множественное: Optional[InclinedMorpherDeclensionMany] = None


def get_inclined_info(data: InclinedMorpherInfo, nominative: str) -> InclinedInfo:
    inclined_info = InclinedInfo(
        nominative=nominative,
        genitive=data.Р,
        dative=data.Д,
        accusative=data.В,
        instrumental=data.Т,
        prepositional=data.П,
    )

    return inclined_info


In [ ]:
import requests

Fullname = str

HeaderPosition = str

InclinedFullname = InclinedInfo

InclinedHeaderPosition = InclinedInfo

InclinedFullnameResult = InclinedFullname | None


def unwrap_inclined_header_position(
    header_position: HeaderPosition,
    default_value: InclinedFullname = InclinedFullname(),
) -> InclinedHeaderPosition:
    return unwrap_inclined_fullname(header_position, default_value)


def unwrap_inclined_fullname(
    fullname: Fullname, default_value: InclinedFullname = InclinedFullname()
) -> InclinedFullname:
    inclined_fullname = incline_fullname(fullname)
    result = inclined_fullname or default_value

    return result


def incline_fullname(fullname: Fullname) -> InclinedFullnameResult:
    try:
        url = "https://ws3.morpher.ru/russian/declension"
        headers = {"User-Agent": "My Python script"}
        params = dict(
            s=fullname,
            format="json",
            # token= "AUTH TOKEN HERE"
        )

        response = requests.get(url=url, params=params, headers=headers)
        is_failed_response = response.status_code != 200
        if is_failed_response:
            print(
                f"Failed response({response.status_code}) to morpher on do incline fullname",
                file=sys.stderr,
            )

            return None

        data = InclinedMorpherInfo.schema().loads(response.text)
        inclined_fullname = get_inclined_info(data, fullname)

        return inclined_fullname
    except Exception:
        print("Unknown error on do incline fullname", file=sys.stderr)

        return None


In [ ]:
test_inclined_fullname = incline_fullname("Петров Петр Петрович")
if test_inclined_fullname is None:
    print("Failed get inclined fullname")
else:
    print(f"Inclined fullname: {test_inclined_fullname}")

print()

test_inclined_fullname_initials = incline_fullname("Петров П.В.")
if test_inclined_fullname_initials is None:
    print("Failed get inclined fullname initials")
else:
    print(f"Inclined fullname initials: {test_inclined_fullname_initials}")


### Ввод данных для отчетов


In [ ]:
from jira_dtos import UserWorklogItemDto


def get_jira_worklogs() -> MappedUserWorklogs:
    worklogs_str = '[{"task_name":"Что есть зло? Чтобы это не было, оно возникает из слабости","task_code":"D2C-1","time_spent_seconds":52200,"start_period_date":1662987567.499,"end_period_date":1663078476.16},{"task_name":"Требуется много таланта и умения, чтобы скрыть свой талант и умение","task_code":"D2C-2","time_spent_seconds":126000,"start_period_date":1662473799.184,"end_period_date":1663047038.527},{"task_name":"Человек - это животное, которое умеет совершать сделки. Ни одна собака не будет меняться костью с другой собакой","task_code":"D2C-3","time_spent_seconds":225000,"start_period_date":1661522651.363,"end_period_date":1662722506.429},{"task_name":"Мы не должны расстраиваться, когда другие скрывают правду от нас, так как мы скрываем правду и от себя тоже","task_code":"D2C-4","time_spent_seconds":115200,"start_period_date":1660831746.787,"end_period_date":1661497110.483},{"task_name":"У каждого человека внутри сидит предатель","task_code":"D2C-10","time_spent_seconds":99000,"start_period_date":1660573388.942,"end_period_date":1660813309.804}]'
    worklogs = UserWorklogItemDto.schema().loads(worklogs_str, many=True)

    return worklogs


In [ ]:
from math import trunc


def zpad(val: float, n: int) -> str:
    bits = str(val).split(".")
    result = "%s" % (bits[0].zfill(n))

    return result


def fmt_number_by_spaces(val: float) -> str:
    result = "{:,}".format(val)
    result = result.replace(",", " ")

    return result


def fmt_number_by_spaces_comma(val: float) -> str:
    result = fmt_number_by_spaces(val).replace(".", ", ")

    return result


def get_work_cost_in_words(exact_work_cost: float) -> str:
    inclined_cost = incline_cost(CostInfo(cost=exact_work_cost))
    if inclined_cost is None:
        return ""

    pennies = int(exact_work_cost % 1 * 100)
    pennies = zpad(pennies, 2)
    formatted_work_cost = fmt_number_by_spaces(int(exact_work_cost))
    result = f"{formatted_work_cost} ({inclined_cost.cost}) {inclined_cost.unit} {pennies} коп."

    return result


def get_total_comma_work_cost(val: float) -> str:
    result = fmt_number_by_spaces_comma(val)
    result = f"{result} руб. НДС не облагается"

    return result


def get_work_info() -> WorkInfo:
    exact_work_cost = 322610.89
    formatted_exact_work_cost = fmt_number_by_spaces(exact_work_cost)
    work_cost = int(exact_work_cost)
    work_cost_in_words = get_work_cost_in_words(exact_work_cost)
    total_spent_hours = 171.5
    formatted_work_cost = fmt_number_by_spaces(work_cost)
    formatted_exact_comma_work_cost = get_total_comma_work_cost(exact_work_cost)
    work_info = WorkInfo(
        exact_work_cost=formatted_exact_work_cost,
        exact_comma_work_cost=formatted_exact_comma_work_cost,
        work_cost=formatted_work_cost,
        work_cost_in_words=work_cost_in_words,
        total_spent_hours=total_spent_hours,
    )

    return work_info


def get_act_info() -> ContractInfo:
    month_name = "сентябрь"
    inclined_month = incline_fullname(month_name)
    inclined_month = (
        inclined_month.genitive if inclined_month is not None else f"{month_name}(!)"
    )
    approval_date = f"<<14>> {inclined_month} 2022 г."
    doc = ContractInfo(approval_date=approval_date, number="14-09-22")

    return doc


def get_supplementary_agreement_info() -> ContractInfo:
    month_name = "август"
    inclined_month = incline_fullname(month_name)
    inclined_month = (
        inclined_month.genitive if inclined_month is not None else f"{month_name}(!)"
    )
    approval_date = f"<<15>> {inclined_month} 2022 г."
    doc = ContractInfo(approval_date=approval_date, number="15-08-22")

    return doc


def get_contract_info() -> ContractInfo:
    month_name = "июль"
    inclined_month = incline_fullname(month_name)
    inclined_month = (
        inclined_month.genitive if inclined_month is not None else f"{month_name}(!)"
    )
    approval_date = f"<<14>> {inclined_month} 2022 г."
    doc = ContractInfo(approval_date=approval_date, number="14-07-22")

    return doc


def get_contracts_info() -> ContractsInfo:
    act = get_act_info()
    contract = get_contract_info()
    supplementary_agreement = get_supplementary_agreement_info()
    work_info = get_work_info()
    contracts_info = ContractsInfo(
        act=act,
        contract=contract,
        supplementary_agreement=supplementary_agreement,
        work_info=work_info,
    )

    return contracts_info


In [ ]:
def get_customer() -> CompanyInfo:
    header_fullname_initials_in_end = unwrap_inclined_fullname("Киетака А.Х.")
    header_fullname_initials_in_front = unwrap_inclined_fullname("А.Х. Киетака")
    header_position = unwrap_inclined_header_position("Генеральный директор")
    company_name = "<<Класс превосходства>>"
    initial_company_name = f"ООО {company_name}"
    info = CompanyInfo(
        header_fullname=header_fullname_initials_in_end,
        header_fullname_initials_in_end=header_fullname_initials_in_end,
        header_fullname_initials_in_front=header_fullname_initials_in_front,
        header_position=header_position,
        company_name=company_name,
        initial_company_name=initial_company_name,
    )

    return info


def get_executor() -> CompanyInfo:
    header_fullname = "Ягами Лайт Саятиро"
    header_fullname_initials_in_end = unwrap_inclined_fullname("Ягами Л.С.")
    header_fullname_initials_in_front = unwrap_inclined_fullname("Л.С. Ягами")
    header_position = unwrap_inclined_header_position("ИП")
    company_name = header_fullname
    initial_company_name = company_name
    info = CompanyInfo(
        header_fullname=header_fullname_initials_in_end,
        header_fullname_initials_in_end=header_fullname_initials_in_end,
        header_fullname_initials_in_front=header_fullname_initials_in_front,
        header_position=header_position,
        company_name=company_name,
        initial_company_name=initial_company_name,
    )

    return info


def get_contract_parties_info() -> ContractPartiesInfo:
    customer = get_customer()
    executor = get_executor()
    info = ContractPartiesInfo(customer=customer, executor=executor)

    return info


In [ ]:
def get_doc_templates() -> JinjaTemplates:
    return [get_act_doc_template(), get_task_doc_template()]


### Формирование отчетов из шаблонов


In [ ]:
def get_template_info(
    template: JinjaTemplate,
    worklogs: MappedUserWorklogs,
    contracts_info: ContractsInfo,
    contract_parties_info: ContractPartiesInfo,
) -> ITemplateInfo:
    is_act_template = isinstance(template, ActJinjaTemplate)
    if is_act_template:
        template_info = ActData(
            doc_template=template,
            contracts_info=contracts_info,
            worklogs=worklogs,
            contract_parties_info=contract_parties_info,
        )

        return template_info

    is_task_template = isinstance(template, TaskJinjaTemplate)
    if is_task_template:
        template_info = TaskData(
            doc_template=template,
            contracts_info=contracts_info,
            worklogs=worklogs,
            contract_parties_info=contract_parties_info,
        )

        return template_info

    raise Exception("Unknown template type. Provide matching for this template type.")


jira_worklogs = get_jira_worklogs()
contracts_info = get_contracts_info()
contract_parties_info = get_contract_parties_info()


In [ ]:
doc_templates = get_doc_templates()
infos = map(
    lambda template: get_template_info(
        template, jira_worklogs, contracts_info, contract_parties_info
    ),
    doc_templates,
)


In [ ]:
for info in infos:
    formatted_tex_file_path = format_tex_file_by_doc_template(info)
    print(formatted_tex_file_path)
    # generated_pdf_file_path = generate_pdf_acts(formatted_tex_file_path)
    # print(f"Generated file path: {generated_pdf_file_path}\n")
